# Multimodal Topic Analysis

In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from glob import glob

image_embedding_path = '../../images/image_topic_information.csv'
image_embeddings_df = pd.read_csv(image_embedding_path)
image_embeddings_df.rename(columns={'embedding': 'image_embedding'}, inplace=True)

pneumonia_df_path = '../../NLP_processing/NER_embeddings/pneumonia_type/radgraph_with_embeddings.csv'
pneumonia_df = pd.read_csv(pneumonia_df_path)

In [44]:
pneumonia_df

,Unnamed: 0.1,subject_id_x,hadm_id,study_id,dicom_id,pneumonia_type,Severe,Unnamed: 0,subject_id_y,report_path,...,technique,comparison,findings,impression,has_comparison,report_length,radgraph_text,extracts,processed_radgraph,embedding
0,0,11146739,20000057,57595001,d46de6bb-6ee08654-dd5f127c-1c339f58-f780e465,other,False,536,11146739,../../../severity_data/report_files/p11/p11146...,...,NaN,none.,there are coarse interstitial markings bilater...,NaN,True,1130,NaN,"{'0': {'text': 'nan', 'entities': {}, 'data_so...",NaN,"[-0.035868603736162186, -0.13423331081867218, ..."
1,1,18110461,20001947,57106576,1f239460-e00a31a5-81bdb260-f2929be7-f7cb2f7d,bacterial,False,3906,18110461,../../../severity_data/report_files/p18/p18110...,...,chest pa and lateral,[REMOVED],cardiac silhouette size is normal. mediastinal...,findings concerning for multifocal pneumonia. ...,True,645,cardiac silhouette size is normal. mediastinal...,{'0': {'text': 'cardiac silhouette size is nor...,cardiac is an anatomy. silhouette modifies car...,"[0.03332185745239258, -0.005314456298947334, -..."
2,2,18346104,20002712,56389963,33125f0b-b2a6d1a2-1408ce03-33729f30-68e6fd64,other,False,4007,18346104,../../../severity_data/report_files/p18/p18346...,...,single portable ap view of the chest.,chest radiograph from [REMOVED].,"compared with the prior radiograph, no signifi...","compared with the prior radiograph, no change ...",True,693,"compared with the prior radiograph, no signifi...",{'0': {'text': 'compared with the prior radiog...,significant modifies change. change modifies o...,"[0.039139412343502045, -0.016893945634365082, ..."
3,3,19669999,20005479,56328195,927ce6d1-2f7acc58-55042c9a-e688c114-129c4009,other,True,4612,19669999,../../../severity_data/report_files/p19/p19669...,...,NaN,NaN,lung volumes are low. a large hiatal hernia ca...,low lung volumes and large hiatal hernia. new ...,False,712,lung volumes are low. a large hiatal hernia ca...,{'0': {'text': 'lung volumes are low . a large...,lung is an anatomy. volumes modifies lung. low...,"[0.058571215718984604, -0.01945420540869236, -..."
4,4,12351713,20007567,59692077,aa33b0f7-a2e3dc48-68b210fb-fb0a6ea5-bd1bf388,other,False,1138,12351713,../../../severity_data/report_files/p12/p12351...,...,pa and lateral chest radiographs,prior outside chest ct from [REMOVED] and ches...,lung volumes are decreased. there is susbtanti...,low lung volumes with substantial bibasilar at...,True,569,lung volumes are decreased. there is susbtanti...,{'0': {'text': 'lung volumes are decreased . t...,lung is an anatomy. volumes modifies lung. dec...,"[0.027617672458291054, -0.013795840553939342, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437,4437,10623647,29991969,59289169,f47aa7aa-8461e734-cc03c9ac-f152661b-4700bd0c,bacterial,False,287,10623647,../../../severity_data/report_files/p10/p10623...,...,chest: frontal and lateral,[REMOVED],bilateral patchy pulmonary opacities appear sl...,patchy bilateral mid to lower lung opacities a...,True,1760,bilateral patchy pulmonary opacities appear sl...,{'0': {'text': 'bilateral patchy pulmonary opa...,bilateral modifies pulmonary. patchy modifies ...,"[0.03257104381918907, -0.026108980178833008, -..."
4438,4438,15116068,29993812,55182265,65d1408e-bc59a65c-0be0fa6d-fb54b613-45abd8c1,bacterial,False,2406,15116068,../../../severity_data/report_files/p15/p15116...,...,single portable semi upright radiograph the ch...,"chest ct: [REMOVED], [REMOVED]",extensive bronchiectasis is again noted in the...,"extensive bilateral bronchiectasis, with super...",True,1100,extensive bronchiectasis is again noted in the...,{'0': {'text': 'extensive bronchiectasis is ag...,extensive modifies bronchiectasis. bronchiecta...,"[0.03614714369177818, -0.008272165432572365, -..."
4439,4439,17025867,29996361,50696726,99011231-5f716ee8-5e61eadd-447b48c4-4ec5255a,bacterial,False,3357,17025867,../../../severity_data/report_files/p17/p17025...,...,NaN,radiograph avail

In [45]:
image_embeddings_df

,study_id,subject_id,dicom_id,image_embedding,topic,topic_probability
0,57106576,18110461,1f239460-e00a31a5-81bdb260-f2929be7-f7cb2f7d,"[-1.0478514432907104, -0.9893836975097656, 0.9...",0,1.000000
1,52444794,15447063,82d06e5f-4f17e47a-3a96a851-d9454252-862ff5be,"[0.14750519394874573, -0.8777117133140564, 1.2...",1,1.000000
2,58791719,13243522,fc215cad-d1060eb4-32e74e2e-15e2c40b-d01e66db,"[-0.6212253570556641, -0.6505540609359741, 1.1...",8,0.982032
3,51779043,11423061,9b1832b0-9cf4f257-fd4bbb42-06957578-0bf69814,"[-0.6631452441215515, -0.5168229341506958, 1.0...",12,0.676550
4,58785779,15379716,120b1593-d73e54d8-6d999909-ce1d93ce-d3b26f46,"[-0.08448687195777893, -1.0461037158966064, 1....",0,1.000000
...,...,...,...,...,...,...
2754,58255867,14236258,0f33dea2-1c4e6245-7b21b568-ef0299e9-03c0863a,"[-0.12456560134887695, -0.08151178061962128, 1...",1,1.000000
2755,59289169,10623647,f47aa7aa-8461e734-cc03c9ac-f152661b-4700bd0c,"[-0.27105075120925903, -0.9785194993019104, 1....",0,1.000000
2756,55182265,15116068,65d1408e-bc59a65c-0be0fa6d-fb54b613-45abd8c1,"[-0.37828612327575684, -0.8936127424240112, 0....",-1,0.000000
2757,50696726,17025867,99011231-5f716ee8-5e61eadd-447b48c4-4ec5255a,"[0.11463172733783722, -0.3663841485977173, 0.7...",1,0.950204


In [47]:
pneumonia_df["dicom_id"] = pneumonia_df["dicom_id"].astype(str)
image_embeddings_df["dicom_id"] = image_embeddings_df["dicom_id"].astype(str)

# Merge the two DataFrames
merged_df = pneumonia_df.merge(image_embeddings_df, on="dicom_id", how="inner")
merged_df.to_csv('merged_intensity_image_radgraph_embeddings')
print("Merged filtered embeddings with pneumonia study data and saved to filtered_embeddings_with_pneumonia_reports.csv!")

merged_df

Merged filtered embeddings with pneumonia study data and saved to filtered_embeddings_with_pneumonia_reports.csv!


,Unnamed: 0.1,subject_id_x,hadm_id,study_id_x,dicom_id,pneumonia_type,Severe,Unnamed: 0,subject_id_y,report_path,...,report_length,radgraph_text,extracts,processed_radgraph,embedding,study_id_y,subject_id,image_embedding,topic,topic_probability
0,1,18110461,20001947,57106576,1f239460-e00a31a5-81bdb260-f2929be7-f7cb2f7d,bacterial,False,3906,18110461,../../../severity_data/report_files/p18/p18110...,...,645,cardiac silhouette size is normal. mediastinal...,{'0': {'text': 'cardiac silhouette size is nor...,cardiac is an anatomy. silhouette modifies car...,"[0.03332185745239258, -0.005314456298947334, -...",57106576,18110461,"[-1.0478514432907104, -0.9893836975097656, 0.9...",0,1.000000
1,6,15447063,20009511,52444794,82d06e5f-4f17e47a-3a96a851-d9454252-862ff5be,bacterial,False,2557,15447063,../../../severity_data/report_files/p15/p15447...,...,806,mild cardiomegaly has been stable compared to ...,{'0': {'text': 'mild cardiomegaly has been sta...,mild modifies cardiomegaly. cardiomegaly is an...,"[0.0257903840392828, -0.0005293460562825203, 0...",52444794,15447063,"[0.14750519394874573, -0.8777117133140564, 1.2...",1,1.000000
2,8,13243522,20013465,58791719,fc215cad-d1060eb4-32e74e2e-15e2c40b-d01e66db,bacterial,False,1530,13243522,../../../severity_data/report_files/p13/p13243...,...,1136,right chest wall port-a-cath ends at the cavoa...,{'0': {'text': 'right chest wall port - a - ca...,right modifies wall. chest modifies wall. wall...,"[0.038594670593738556, -0.015855595469474792, ...",58791719,13243522,"[-0.6212253570556641, -0.6505540609359741, 1.1...",8,0.982032
3,9,11423061,20014114,51779043,9b1832b0-9cf4f257-fd4bbb42-06957578-0bf69814,bacterial,True,670,11423061,../../../severity_data/report_files/p11/p11423...,...,685,low lung volumes limit assessment of the lung ...,{'0': {'text': 'low lung volumes limit assessm...,low is located at volumes. lung is an anatomy....,"[0.030176309868693352, -0.024805540218949318, ...",51779043,11423061,"[-0.6631452441215515, -0.5168229341506958, 1.0...",12,0.676550
4,10,15379716,20015580,58785779,120b1593-d73e54d8-6d999909-ce1d93ce-d3b26f46,bacterial,False,2511,15379716,../../../severity_data/report_files/p15/p15379...,...,600,NaN,"{'0': {'text': 'nan', 'entities': {}, 'data_so...",NaN,"[-0.035868603736162186, -0.13423331081867218, ...",58785779,15379716,"[-0.08448687195777893, -1.0461037158966064, 1....",0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,4435,14236258,29989743,58255867,0f33dea2-1c4e6245-7b21b568-ef0299e9-03c0863a,bacterial,False,1992,14236258,../../../severity_data/report_files/p14/p14236...,...,695,vague opacity projecting over the right mid/lo...,{'0': {'text': 'vague opacity projecting over ...,vague modifies opacity. opacity is located at ...,"[0.04184424504637718, -0.01704668439924717, 0....",58255867,14236258,"[-0.12456560134887695, -0.08151178061962128, 1...",1,1.000000
2592,4437,10623647,29991969,59289169,f47aa7aa-8461e734-cc03c9ac-f152661b-4700bd0c,bacterial,False,287,10623647,../../../severity_data/report_files/p10/p10623...,...,1760,bilateral patchy pulmonary opacities appear sl...,{'0': {'text': 'bilateral patchy pulmonary opa...,bilateral modifies pulmonary. patchy modifies ...,"[0.03257104381918907, -0.026108980178833008, -...",59289169,10623647,"[-0.27105075120925903, -0.9785194993019104, 1....",0,1.000000
2593,4438,15116068,29993812,55182265,65d1408e-bc59a65c-0be0fa6d-fb54b613-45abd8c1,bacterial,False,2406,15116068,../../../severity_data/report_files/p15/p15116...,...,1100,extensive bronchiectasis is again noted in the...,{'0': {'text': 'extensive bronchiectasis is ag...,extensive modifies bronchiectasis. bronchiecta...,"[0.03614714369177818, -0.008272165432572365, -...",55182265,15116068,"[-0.37828612327575684, -0.8936127424240112, 0....",-1,0.000000
2594,4439,17025867,29996361,50696726,99011231-5f716ee8-5e61eadd-447b48c4-4ec5255a,bacterial,False,3357,17025867,../../../severity_data/report_files/p17/p17025..

In [ ]:
import pandas as pd
import numpy as np
import ast

# Load the merged dataset
merged_df = pd.read_csv("merged_intensity_image_radgraph_embeddings")
def convert_to_array(embedding_str):
    try:
        return np.array(ast.literal_eval(embedding_str), dtype=np.float32)
    except (ValueError, SyntaxError):
        return None  # Return None for rows with invalid embeddings

# Apply to both text and image embeddings
merged_df["text_embedding"] = merged_df["embedding"].apply(convert_to_array)
merged_df["image_embedding"] = merged_df["image_embedding"].apply(convert_to_array)

# Drop rows with invalid embeddings
merged_df = merged_df.dropna(subset=["text_embedding", "image_embedding"])

# Convert 'radgraph_text' to string and handle missing values
merged_df["radgraph_text"] = merged_df["radgraph_text"].fillna("").astype(str)
docs = merged_df["radgraph_text"].tolist()
# Debugging: Ensure all values are strings
if not all(isinstance(doc, str) for doc in docs):
    raise ValueError("ERROR: Some values in `docs` are not strings!")

print(f"Successfully extracted {len(docs)} valid text documents.")

text_embeddings = np.vstack(merged_df["text_embedding"].values)
image_embeddings = np.vstack(merged_df["image_embedding"].values)

# Concatenate text and image embeddings to form multimodal embeddings
multimodal_embeddings = np.concatenate([text_embeddings, image_embeddings], axis=1)

print(f"Multimodal embeddings shape: {multimodal_embeddings.shape}")  # Should be (num_docs, text_dim + image_dim)

# Ensure consistency
assert len(docs) == multimodal_embeddings.shape[0], "Mismatch between documents and multimodal embeddings!"

✅ Successfully extracted 2596 valid text documents.
Multimodal embeddings shape: (2596, 2144)


In [32]:
from bertopic import BERTopic
from bertopic.representation import VisualRepresentation, KeyBERTInspired, MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import os
import torch
from transformers import AutoTokenizer, AutoModel

# Load the previously saved CXR model
model_path = "../../models/RADGRAPH_embeddings/bacterial"
tokenizer = AutoTokenizer.from_pretrained(model_path)
embedding_model = AutoModel.from_pretrained(model_path)

# Add a visual representation model to handle images
visual_model = VisualRepresentation()

# UMAP for dimensionality reduction
umap_model = UMAP(n_neighbors=30, n_components=5, min_dist=0.0, metric='cosine')

# HDBSCAN for clustering
hdbscan_model = HDBSCAN(min_samples=20, 
                        gen_min_span_tree=True, 
                        prediction_data=True, 
                        min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='leaf')

# Vectorizer for text processing
vectorizer_model = CountVectorizer(strip_accents='unicode', 
                                   stop_words='english', 
                                   ngram_range=(1, 3), 
                                   max_df=0.6)  # Remove overly general terms

# Class-based TF-IDF model
ctfidf_model = ClassTfidfTransformer(bm25_weighting=False, reduce_frequent_words=True)

# Representation model (for interpreting topics)
representation_model = {
    "KeyBERT": KeyBERTInspired(top_n_words=30, random_state=42), 
    "Diversity": MaximalMarginalRelevance(diversity=0.7),
    "Visual_Aspect": visual_model  # Adds image-based topic representation
}

# Initialize BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    ctfidf_model=ctfidf_model, 
    vectorizer_model=vectorizer_model,
    verbose=True,
    representation_model=representation_model,
    top_n_words=10,
    calculate_probabilities=True,  # Ensures probability calculations for topics
)

# Fit the model with multimodal embeddings (text + image)
topics, probabilities = topic_model.fit_transform(docs, embeddings=multimodal_embeddings)

# Save the topic model
topic_model.save("../../models/BERTopic_Bacterial/representations/multimodal_representation", 
                 serialization="pytorch", save_ctfidf=True)

2025-02-13 02:16:05,047 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-13 02:16:15,607 - BERTopic - Dimensionality - Completed ✓
2025-02-13 02:16:15,608 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-13 02:16:15,742 - BERTopic - Cluster - Completed ✓
2025-02-13 02:16:15,744 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|          | 0/27 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
import base64
from io import BytesIO
from IPython.display import HTML

# Extract topic info
df_topics = topic_model.get_topic_info()

# Convert images to base64 for HTML display
def image_base64(im_path):
    with open(im_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode()

def image_formatter(im_path):
    return f'<img src="data:image/jpeg;base64,{image_base64(im_path)}" width="150">'

# Ensure we have image paths in our dataframe
merged_df["image_path"] = merged_df["dicom_id"].apply(lambda x: f"../../severity_data/image_files/{x}.jpg")

# Match each topic with its most representative image
df_topics["Visual_Aspect"] = df_topics["Topic"].map(
    lambda topic: merged_df[merged_df["topic"] == topic]["image_path"].values[0] if topic in merged_df["topic"].values else None
)

# Display topic info with representative images
HTML(df_topics.to_html(formatters={'Visual_Aspect': image_formatter}, escape=False))


In [50]:
import os
import pandas as pd
import numpy as np
import glob  # To handle multiple DICOM images per study

# Load dataset
merged_df = pd.read_csv("merged_intensity_image_radgraph_embeddings")

# Function to construct the correct image path
def find_dicom_path(subject_id, study_id):
    base_path = "../../severity_data/image_files"
    subject_folder = f"p{str(subject_id)[:2]}/{subject_id}/{study_id}"
    full_path = os.path.join(base_path, subject_folder)

    # Find all DICOM files in the directory
    dicom_files = glob.glob(os.path.join(full_path, "*.dcm"))
    
    # If multiple DICOMs exist, return the first one (or modify logic as needed)
    return dicom_files[0] if dicom_files else None

# Apply function to generate valid DICOM paths
merged_df["image_path"] = merged_df.apply(lambda row: find_dicom_path(row["subject_id_x"], row["study_id_x"]), axis=1)

# Drop rows where no image path was found
merged_df = merged_df.dropna(subset=["image_path"])

# Debugging: Show how many images were found
print(f"Found {len(merged_df)} valid DICOM images linked to reports!")

# Save updated dataset
merged_df.to_csv("filtered_merged_with_image_paths.csv", index=False)
print("Saved filtered dataset with correct DICOM image paths.")

In [ ]:
severity_data/image_files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.tfrecord